# Simulation template.

In this notebook we produce the results of the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 641 ms, sys: 56 ms, total: 697 ms
Wall time: 695 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = "/home/linneu/ssh_csvs/ssh1/periodic_100_6561.csv"   
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = "/home/linneu/ssh_simulations/ssh1/periodic_100_6561"
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/linneu/ssh_csvs/ssh2/periodic_220_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/linneu/ssh_simulations/ssh2/periodic_220_6561"
random_state = 657


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/linneu/anaconda3/envs/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443420 entries, 0 to 1443419
Columns: 228 entries, id to feat219
dtypes: float64(223), int32(3), object(2)
memory usage: 2.4+ GB
CPU times: user 44.1 s, sys: 4.22 s, total: 48.4 s
Wall time: 49.2 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  220
n_hamiltonians:  6561
n_ts:  2
CPU times: user 1.36 ms, sys: 131 µs, total: 1.49 ms
Wall time: 1.19 ms


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.8414875781130925
% val:  0.0
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  5521
number of val hamiltonians:  0
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [01:40<2:46:20, 100.82s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [03:18<2:43:15, 99.96s/it] 

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [2:46:26<00:00, 101.30s/it]

running experiments: 100%|██████████| 100/100 [2:46:26<00:00, 99.86s/it] 

CPU times: user 1d 12h 15min 54s, sys: 3min 44s, total: 1d 12h 19min 39s
Wall time: 2h 46min 26s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.130846,-0.300844,...,0.080274,0.115552,0.174807,-0.244872,-0.319478,0.059064,0.000568,0.244891,0.300844,-0.130846
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.130846,-0.300844,...,-0.080274,0.115552,-0.174807,-0.244872,0.319478,0.059064,-0.000568,0.244891,-0.300844,-0.130846
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.139243,-0.040832,...,0.177651,-0.085976,-0.069808,-0.109102,-0.140759,0.184935,0.184071,-0.052390,-0.040832,-0.139243
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.048483,0.015611,...,-0.065026,0.044350,0.035714,0.029502,0.046097,-0.070102,-0.070512,0.025241,0.015611,0.048483
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,-0.005059,0.009352,...,-0.013734,0.009909,0.002329,0.008250,0.011761,-0.009332,-0.002720,-0.004584,-0.009352,0.005059
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.050074,-0.011117,...,0.049582,0.005255,0.001717,-0.054642,-0.055678,0.050847,0.051142,-0.002840,-0.011117,-0.050074
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,2,test,0.009938,-0.026818,...,-0.013334,0.022656,0.023129,-0.013172,-0.023569,-0.006879,-0.004162,0.025408,0.026818,-0.009938
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.014007,0.025469,...,-0.039242,0.030036,0.008341,0.021972,0.032295,-0.027020,-0.008046,-0.011692,-0.025469,0.014007
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,-1,test,-0.053983,-0.019696,...,0.075379,-0.071666,-0.058670,-0.017526,-0.045907,0.086976,0.089522,-0.040115,-0.019696,-0.053983
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.068700,0.013277,...,-0.056877,-0.023603,-0.014327,0.078240,0.077328,-0.060835,-0.062904,-0.003322,0.013277,0.068700


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 402 ms, sys: 2 µs, total: 402 ms
Wall time: 399 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
181678,825,H_-1.5_-1.25_-1.02171.dat,-1.50,-1.25,-1.021710,-1,-1,test,-0.023412,-0.057631,...,-0.006920,0.100712,-0.025870,0.107607,-0.060230,0.043301,-0.057674,0.004203,-0.057631,-0.023412
150514,684,H_-1.6_-0.2_-0.00232415.dat,-1.60,-0.20,-0.002324,0,0,train,0.020604,-0.067427,...,0.074341,-0.052840,-0.020556,-0.056978,-0.072613,0.043246,0.010467,0.057502,0.067427,-0.020604
713104,3241,H_0_-1.95_1.00271.dat,0.00,-1.95,1.002712,1,1,train,0.054443,0.003672,...,-0.021032,-0.003205,0.075338,0.026123,0.020282,0.044712,-0.123146,-0.081737,-0.003672,-0.054443
135475,615,H_-1.65_0.4_-0.000978776.dat,-1.65,0.40,-0.000979,0,0,val,-0.000901,-0.002173,...,0.045434,-0.026597,-0.023490,0.009621,0.011508,-0.004692,-0.002071,-0.001771,-0.002173,-0.000901
245832,1117,H_-1.35_1.2_2.12698e-05.dat,-1.35,1.20,0.000021,0,0,train,-0.090432,-0.090284,...,0.001029,-0.088619,0.122490,0.078463,-0.080944,0.013039,0.017511,-0.026110,0.090284,0.090432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962902,4376,H_0.7_-1.9_1.00171.dat,0.70,-1.90,1.001710,1,1,train,-0.017088,0.002044,...,0.060399,-0.048990,-0.050452,-0.017258,0.008899,0.074440,-0.005919,-0.045190,-0.002044,0.017088
789016,3586,H_0.2_-0.9_1.00256.dat,0.20,-0.90,1.002564,1,1,val,0.013358,0.022503,...,-0.007885,-0.060954,0.121620,0.094747,-0.023431,0.056776,-0.022553,-0.085041,0.022503,0.013358
168580,766,H_-1.55_-0.15_-0.00220794.dat,-1.55,-0.15,-0.002208,0,0,val,0.079618,0.079803,...,0.078408,0.075226,0.078652,0.072165,0.079936,0.079866,0.079353,0.079754,0.079803,0.079618
328877,1494,H_-1.1_-0.2_-0.00301281.dat,-1.10,-0.20,-0.003013,0,0,train,-0.117496,-0.108835,...,0.040133,0.081797,-0.053611,-0.087614,0.082583,0.095876,-0.083429,-0.093431,0.108835,0.117496


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.4208199969516842
% val:  0.42066758116140834
% test:  0.15851242188690748
% train + val + test:  1.0


number of train hamiltonians:  2761
number of val hamiltonians:  2760
number of test hamiltonians:  1040
total number of hamiltonians:  6561


train ids:  [5519, 5705, 1622, 2082, 3968, 5507, 2106, 3369, 6468, 4140, 3487, 5593, 2775, 4576, 3212, 4969, 4537, 5966, 6184, 6190, 5440, 3079, 2318, 6123, 4962, 6117, 352, 1004, 1828, 4353, 6240, 1987, 961, 4395, 46, 77, 157, 5316, 6469, 4629, 1295, 508, 4191, 959, 4983, 1975, 1346, 6057, 6221, 2252, 5522, 2905, 4959, 2151, 5948, 858, 3913, 4709, 3284, 311, 5186, 4706, 3753, 1438, 5470, 5366, 3305, 1129, 6039, 4205, 771, 4290, 3416, 4052, 3119, 61, 5222, 435, 1432, 612, 2630, 3331, 684, 1918, 2725, 6051, 5787, 3978, 3250, 4850, 4827, 1422, 2915, 199, 3920, 708, 3801, 5895, 2579, 4798, 4199, 2155, 4780, 3121, 1372, 4196, 3546, 4878, 2801, 5772, 2772, 5642, 3099, 1115, 479, 4517, 1178, 3446, 5550, 2267, 4689, 4249, 3912, 3182, 3252,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 335 µs, sys: 1e+03 ns, total: 336 µs
Wall time: 216 µs


,t1,t2,type_of,-1,0,1,2,phase,pred_phase
id,,,,,,,,,
0,-2.0,-2.00,test,0.354545,0.290909,0.218182,0.136364,999,-1
1,-2.0,-1.95,test,0.331818,0.340909,0.195455,0.131818,-1,0
2,-2.0,-1.90,test,0.395455,0.309091,0.190909,0.104545,-1,-1
3,-2.0,-1.85,test,0.381818,0.295455,0.227273,0.095455,-1,-1
4,-2.0,-1.80,test,0.400000,0.286364,0.227273,0.086364,-1,-1
...,...,...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.504545,0.190909,0.059091,0.245455,-1,-1
6557,2.0,1.85,test,0.522727,0.163636,0.081818,0.231818,-1,-1
6558,2.0,1.90,test,0.413636,0.245455,0.045455,0.295455,-1,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1443420


,id,phase,pred_phase,type_of
0,0,999,2,test
1,0,999,-1,test
2,0,999,2,test
3,0,999,1,test
4,0,999,2,test
...,...,...,...,...
1443415,6560,999,1,test
1443416,6560,999,1,test
1443417,6560,999,1,test
1443418,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9997761022027593,
 'eigenvector_val': 0.9588455204216074,
 'eigenvector_test': 0.6337813525240766,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 0.9971014492753624,
 'hamiltonian_test': 0.9045401629802096}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}